In [1]:
import numpy as np
import scipy
import scipy.stats

import matplotlib.pyplot as plt

%matplotlib inline

from tqdm.auto import tqdm
import string

# Substitution Cipher

In this task we will decrypt data that was scrambled using a Substitution Cipher. We assume that encryption key is unknown and we want to decrypt the data and read the code using recovered decryption key. [Introduction from here](http://statweb.stanford.edu/~cgates/PERSI/papers/MCMCRev.pdf) gives reference to the original task.

As verification we will take a piece from "Alice's adventures in Wonderland". We scramble data with a random encryption key, which we forgot after encrypting, and we would like to decrypt this encrypted text using MCMC Chains.

In [2]:
plain_text = """
Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?'
So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her.
There was nothing so very remarkable in that; nor did Alice think it so very much out of the way to hear the Rabbit say to itself, 'Oh dear! Oh dear! I shall be late!' (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the Rabbit actually took a watch out of its waistcoat-pocket, and looked at it, and then hurried on, Alice started to her feet, for it flashed across her mind that she had never before see a rabbit with either a waistcoat-pocket, or a watch to take out of it, and burning with curiosity, she ran across the field after it, and fortunately was just in time to see it pop down a large rabbit-hole under the hedge.
In another moment down went Alice after it, never once considering how in the world she was to get out again.
The rabbit-hole went straight on like a tunnel for some way, and then dipped suddenly down, so suddenly that Alice had not a moment to think about stopping herself before she found herself falling down a very deep well.
"""

We will use 26 letters of English alphabet.

In [3]:
characters = string.ascii_lowercase
characters_dict = {char : c for c, char in enumerate(characters, start=1)}
m = len(characters) + 1

## Generate random encryption key.

Here are functions that will be used to encrypt/decrypt text.

In [4]:
def encode_text(text_to_encode, characters_dict):
    """This function turns a text string into an integer sequence using given dictionary."""
    characters_set = set(characters_dict.keys())
    return np.r_[[characters_dict[char] if char in characters_set else 0 for char in text_to_encode.strip().lower()]]

def decode_text(text_to_decode, characters):
    """This function turns an integer sequence into a text string using given list of characters."""
    characters_array = np.array([" "] + list(characters))
    return "".join(characters_array[text_to_decode])

def apply_cipher(text_as_int_array, cipher):
    "This function applies substitution cipher to an integer sequence."
    return cipher[text_as_int_array]

Generate encryption and decryption keys. They are just permutations of the alphabet.

In [5]:
np.random.seed(1234)

encryption_indices = np.random.permutation(np.arange(m-1))
decryption_indices = np.argsort(encryption_indices)

characters_array = np.array(list(characters))
encryption_key = "".join(characters_array[encryption_indices])
decryption_key = "".join(characters_array[decryption_indices])

Check encoding/decoding functions and encryption/decryption keys.

In [6]:
encryption_key_encoded = np.r_[0, encode_text(encryption_key, characters_dict)]
decryption_key_encoded = np.r_[0, encode_text(decryption_key, characters_dict)]

text = "The quick brown fox jumps over the lazy dog"

encoded_text = encode_text(text, characters_dict)
cipher_text = apply_cipher(encoded_text, encryption_key_encoded)
encoded_text = apply_cipher(cipher_text, decryption_key_encoded)
decoded_text = decode_text(encoded_text, characters_dict)
decoded_text

'the quick brown fox jumps over the lazy dog'

Encrypt cipher text.

In [7]:
plain_text_encoded = encode_text(plain_text, characters_dict)
cipher_text = apply_cipher(plain_text_encoded, encryption_key_encoded)

## Collect frequences 

Collect frequences of two character combinations (bigrams) over large text corpus and from encrypted text. We will store them in a matrix and interpret it as a transition matrix: from the first character to the second.

In [8]:
def collect_transition_frequences(data, transition_matrix):
    """For a given integer sequence, which corresponds to some char sequence, 
       return transitions for adjacent values."""
    transitions = data.repeat(2)[1:-1].reshape(-1, 2)
    for i, j in transitions:
        transition_matrix[i, j] += 1
    
    return transition_matrix

def collect_empirical_frequences(filename, characters_dict, m):
    """Collect frequences over large text corpus, return transition matrix."""
    transition_matrix = np.zeros((m, m))
    with open(filename) as f:
        for line in f:
            line_encoded = encode_text(line, characters_dict)
            if line_encoded.size > 1:
                transition_matrix = collect_transition_frequences(line_encoded, transition_matrix)
                
    return transition_matrix

def collect_observed_frequences(cipher_text, characters_dict, m):
    """Collect frequences over encrypted text, return nonzero indices of 
       transition matrix for both dimentions and values for those indices.
       `values = transition_matrix[indices_1, indices_2]`"""
    transition_matrix = np.zeros((m, m))
    transition_matrix = collect_transition_frequences(cipher_text, transition_matrix)
    
    return transition_matrix

Collect frequences.

In [9]:
empirical_frequences = collect_empirical_frequences('war_and_peace.txt', characters_dict, m)
observed_frequences = collect_observed_frequences(cipher_text, characters_dict, m)

## General algorithm

Our Chain will include states that are permutations of the substitution cipher. Algorithm has following steps:

1. Start by picking up a random current state. 
2. Create a proposal for a new state by swapping two or more random letters in the current state.
3. Use a Scoring Function which calculates the score of the current state $Score_{old}$ and the proposed state $Score_{new}$.
4. If the score of the proposed state is more than current state, Move to Proposed State.
5. Else flip a coin which has a probability of Heads $\frac{Score_{new}}{Score_{old}}$  . If it comes heads move to proposed State.
6. Repeat from Step 2.

We want ot reach a steady state where the chain has the stationary distribution of the needed states. This state of chain could be used as a solution.

Your goal is to implement steps 2 and 3 (see the templates below).

## Step 2: Prepare sampling function.



To generate a new proposed cipher we randomly select several positions and swap values at those positions. It corresponds to change in seveal mappings of encrypted characters in decrypted ones. Example with 2 swaps.

was|now
-|-
A -> B | A -> B
B -> C | B -> C
C -> D | C -> A
D -> A | D -> D

In [10]:
def generate_cipher(cipher, m, size=2):
    """Swap two or more random positions in cipher.
        
        cipher, np.array - current mapping from value(int) in encrypted text (index of array cell) into value(int) in decrypted text(value of array cell).
        m, int - capacity of used alphabet,
        size, int - number of positions to change.
    """
    
    # Your code here
    ids_to_change = list(range(len(cipher)))
    new_cipher = cipher.copy()
    
    while size > 0 and len(ids_to_change) > 1:
        ix1, ix2 = np.random.choice(ids_to_change, 2)
        new_cipher[[ix1, ix2]] = new_cipher[[ix2, ix1]]
        ids_to_change = [i for i in ids_to_change if i not in [ix2, ix1]]
        size -= 1
    
    return new_cipher

## Step 3: Prepare scoring function.

We want to use a scoring function for each state(Decryption key) which assigns a positive score to each decryption key. This score intuitively should be larger if the encrypted text looks more like actual english, when decrypted using this decryption key. We will check a large text and calculate frequences: how many times one character comes after another in a large text like "War and Peace".

For each pair of characters $\beta_1$ and $\beta_2$ (e.g. $\beta_1$ = A and $\beta_2$ = B), we let $R(\beta_1,\beta_2)$ record the number of times that specific pair(e.g. "AB") appears consecutively in the reference text.

Similarly, for a considered decryption key $x$, we let $F_x(\beta_1,\beta_2)$ record the number of times that
pair appears when the cipher text is decrypted using the decryption key $x$.

We then Score a particular decryption key $x$ using:

$$Score(x) = \prod R(\beta_1,\beta_2)^{F_x(\beta_1,\beta_2)}$$
    
To make life easier with calculations we will calculate $log(Score(x))$

Now, you need to implement scoring function. As input it takes 
- `cipher`: mapping between encrypted characters and decrypted characters,
- `observed_frequences`: transition matrix for cipher text, matrix representation of $F_x(\beta_1,\beta_2)$,
- `empirical_frequences`: transition matrix for large text, matrix representation of $R(\beta_1,\beta_2)$.

Scoring function returns $log(Score(x))$. You need correctly process zero values in transition matrices while calculating the score.

In [11]:
def score_cipher(cipher, observed_frequences, empirical_frequences):
    
    # Your code here
    score = 0
    for i in range(len(cipher)):
        for j in range(len(cipher)):
            R_freq = empirical_frequences[i, j]
            F_freq = observed_frequences[cipher[i], cipher[j]]
            score += F_freq * np.log(R_freq + 1) #add 1 to avoid log(0)
    
    return score

**Note from Andrei Moisenko:** here I test scoring function, passing 1- and 15-step cipher to un-encoded text to see score difference.

In [12]:
test_cipher_real = np.arange(m)
test_cipher_1step = generate_cipher(test_cipher_real, m, size=1)
test_cipher_15step = generate_cipher(test_cipher_real, m, size=15)

test_observed_frequences_real = collect_observed_frequences(encoded_text, characters_dict, m)

score_1step = score_cipher(test_cipher_1step, test_observed_frequences_real, empirical_frequences)
score_15step = score_cipher(test_cipher_15step, test_observed_frequences_real, empirical_frequences)

print(f'Score after single permutation:\t{score_1step:.0f}')
print(f'Score after 15 permutations:\t{score_15step:.0f}')

Score after single permutation:	356
Score after 15 permutations:	252


**Note from Andrei Moisenko:** as expected, the more cipher is different from the true one, the less the score is. Seems like our scoring function works correctly.

## Decryption

Now we a ready to decrypt cipher text.

In [13]:
def decrypting(observed_frequences, empirical_frequences, n_iters, m, step_size, seed, print_it=1000):
    """This function finds most suited decrypting cipher(1D np.array).
        observed_frequences, 2D np.array - transition matrix with frequences for cipher text,
        empirical_frequences, 2D np.array - transition matrix with frequences for large text,
        n_iters, int - number of MCMC iterations,
        step_size, int - number of changes in cipher per one iteration,
        seed, int - seed for random generator,
        print_it, int - print decrypted text every `print_it` iterations.
    """

    np.random.seed(seed)

    # 1. Start by picking up a random current state. 
    cipher_old = np.arange(m)
    score_cipher_old = score_cipher(cipher_old, observed_frequences, empirical_frequences)
    best_state, score = cipher_old, score_cipher_old

    for i in tqdm(range(1, n_iters+1)):

        # 2. Create a proposal for a new state by swapping two or more random letters in the current state.
        cipher_new = generate_cipher(cipher_old, m, size=step_size)

        # 3. Use a Scoring Function which calculates the score of the current state $Score_{old}$ and the proposed State $Score_{new}$.
        score_cipher_new = score_cipher(cipher_new, observed_frequences, empirical_frequences)
        acceptance_probability = np.min((1, np.exp(score_cipher_new - score_cipher_old)))

        # 4. If the score of the proposed state is more than current state, Move to Proposed State.
        # 5. Else flip a coin which has a probability of Heads $Score_{new}/Score_{old}$. If it comes heads move to proposed State.
        if score_cipher_old > score:
            best_state, score = cipher_old, score_cipher_old
            
        if acceptance_probability > np.random.uniform(0,1):
            cipher_old, score_cipher_old = cipher_new, score_cipher_new
        if i % print_it == 0:
            print(f"iter {i}: {decode_text(apply_cipher(cipher_text[0:99], cipher_old), characters)}")

    return best_state

**Note from Andrei Moisenko:** decrypting function contained several errors and inefficiencies, so I corrected it

In [14]:
def decrypting_correct(observed_frequences, empirical_frequences, n_iters, m, step_size, seed, print_it=1000):
    """This function finds most suited decrypting cipher(1D np.array).
        observed_frequences, 2D np.array - transition matrix with frequences for cipher text,
        empirical_frequences, 2D np.array - transition matrix with frequences for large text,
        n_iters, int - number of MCMC iterations,
        step_size, int - number of changes in cipher per one iteration,
        seed, int - seed for random generator,
        print_it, int - print decrypted text every `print_it` iterations.
    """

    np.random.seed(seed)
    
    best_state = np.arange(m)
    best_score = score_cipher(best_state, observed_frequences, empirical_frequences)

    for i in tqdm(range(1, n_iters+1)):

        cipher_new = generate_cipher(best_state, m, size=step_size)
        score_cipher_new = score_cipher(cipher_new, observed_frequences, empirical_frequences)
        acceptance_prob = min(1, np.exp(score_cipher_new - best_score))
        
        if  np.random.choice([True, False], p=[acceptance_prob, 1-acceptance_prob]):
            best_state, best_score = cipher_new, score_cipher_new
            
        if i % print_it == 0 or i==1:
            print(f"iter {i}: {decode_text(apply_cipher(cipher_text[0:99], best_state), characters)}", round(best_score))

    return best_state

**Note from Andrei Moisenko:** I greatly increased number of iterations and still current MCMC decoder fails to solve the task even in 8.5 minutes and 500 000 iterations.

In [15]:
decrypt_cipher = decrypting_correct(observed_frequences, empirical_frequences, 500000, m, 4, 345, 10000)

print(
    f"\nDecoded Text: {decode_text(apply_cipher(cipher_text, decrypt_cipher), characters)}\n\n"
    f"MCMC KEY  : {''.join(characters_array[decrypt_cipher[1:]-1])}\n"
    f"ACTual KEY: {decryption_key}"
)

  0%|          | 0/500000 [00:00<?, ?it/s]

iter 1: hpacb vhr qbfayyayf zw fbz dbio zaibm ws razzayf qo ebi rarzbi wy zeb qhyn  hym ws ehdayf ywzeayf z 11002


/var/folders/sv/2km52mqj29qc51s4p_b4vq4c0000gn/T/ipykernel_34413/4028872446.py:20: RuntimeWarning: overflow encountered in exp
  acceptance_prob = min(1, np.exp(score_cipher_new - best_score))


iter 10000: elhvs fex gsrhnnhnr mo rsm ksiz mhisd oj xhmmhnr gz bsi xhxmsi on mbs geny  end oj bekhnr nombhnr m 15548
iter 20000: eihsc nej pcvhmmhmv lk vcl ocgz lhgcd kr jhllhmv pz bcg jhjlcg km lbc pemx  emd kr beohmv mklbhmv l 15775
iter 30000: eihnc vej pcmhsshsm gk mcg qclz ghlcd kr jhgghsm pz bcl jhjgcl ks gbc pesy  esd kr beqhsm skgbhsm g 15838
iter 40000: eihvc nej pcshmmhms ok sco qclz ohlcd kr jhoohms pz bcl jhjocl km obc pemy  emd kr beqhms mkobhms o 15847
iter 50000: eihkc sej pcnhmmhmn ov nco qclz ohlcd vr jhoohmn pz bcl jhjocl vm obc pemy  emd vr beqhmn mvobhmn o 15860
iter 60000: eghcv mej pvthsshst ik tvi qvlz ihlvd kr jhiihst pz bvl jhjivl ks ibv pesy  esd kr beqhst skibhst i 15961
iter 70000: eghnm kej pmshtthts ov smo qmlz ohlmd vr jhoohts pz bml jhjoml vt obm pety  etd vr beqhts tvobhts o 16020
iter 80000: eghfm kej pmshtthts ov smo qmlz ohlmd vr jhoohts pz bml jhjoml vt obm pety  etd vr beqhts tvobhts o 16048
iter 90000: eghcm oej pmshtthts lv sml qmkz lhkmd vr jhl

## Working Solution  
_By Andrei Moisenko_

I could just leave this failed decoder as it is, but it was interesting for me to try creating a working MCMC decoder using the same bigrams idea.  
  
Major differences to the approach above:  
- empirical frequences are normalized, which helps more smooth and fast training
- hash tables instead of arrays to improve code time complexity
- random cifer initiation for MCMC process

In [20]:
import random, re, string
import numpy as np


class EnigmaForDummies:
    def __init__(self) -> None:
        self.emp_freq_prepared = False
        self.regex_pattern = re.compile("[^a-z ]")
        self.alphabet = " " + string.ascii_lowercase
        self.char_index_mapping = dict(
            zip(list(self.alphabet), range(len(self.alphabet)))
        )

    def encrypt_or_decrypt(self, text: str, mapping: dict) -> str:
        """Transform text using mapping dictionary

        Args:
            text (str): text_to_transform
            mapping (dict): mapping dictionary for symbols transformation

        Returns:
            str: transformed text
        """
        return "".join([mapping[s] for s in text])

    def prepare_empirical_freq_normalized(self, path_to_file: str) -> None:
        """Prepare dictionary of normalized empirical frequences from provided corpus

        Args:
            path_to_file (str): path to corpus text file
        """
        # Dictionary to store bigram pairs counts
        char_bigram_counts = {}

        with open(path_to_file, encoding="utf-8") as f:
            for line in f:
                # pre-process line
                clean_line = np.array(list(self.regex_pattern.sub("", line.lower())))

                # create symbols transition array
                transitions = clean_line.repeat(2)[1:-1].reshape(-1, 2)

                # count every symbols transition met in line
                for i, j in transitions:
                    char_bigram_counts[(i, j)] = (
                        char_bigram_counts.setdefault((i, j), 0) + 1
                    )

        # create letters encoder
        letters = [" "] + list(string.ascii_lowercase)
        unigram_to_index = dict(zip(letters, range(len(letters))))

        # Create transition matrix
        n = len(unigram_to_index)
        transition_matrix = np.ones((n, n)) + 1

        # fill in transition matrix for each pair
        for s_pair in char_bigram_counts.keys():
            transition_matrix[unigram_to_index[s_pair[0]]][
                unigram_to_index[s_pair[1]]
            ] = char_bigram_counts[s_pair]

        # normalize matrix rows
        row_sums = transition_matrix.sum(axis=1)
        self.empirical_frequences = transition_matrix / row_sums[:, np.newaxis]
        self.emp_freq_prepared = True

    def encrypt_text(self, text: str) -> str:
        """Perform encryption of text by randomly mapping each letter to other letter in alphabet

        Args:
            text (str): input text to encrypt

        Returns:
            str: encrypted text
        """
        text_cleaned = self.regex_pattern.sub("", text.lower())
        shuffled_alphabet = list(self.alphabet)
        random.shuffle(shuffled_alphabet)
        random_cifer = dict(zip(list(self.alphabet), shuffled_alphabet))
        encrypted_text = self.encrypt_or_decrypt(text_cleaned, random_cifer)
        return encrypted_text

    def score_cipher(self, cipher: dict, encrypted: str) -> float:
        """Score current cipher given encrypted text. The higher score the better

        Args:
            cipher (dict): proposed cipher to score
            encrypted (str): encrypted text

        Returns:
            (float or str): cipher score or decrypted sample
        """
        decrypted = self.encrypt_or_decrypt(
            encrypted, {v: k for k, v in cipher.items()}
        )
        
        score = 0
        for i in range(len(decrypted) - 1):
            first_symbol = self.char_index_mapping[decrypted[i]]
            second_symbol = self.char_index_mapping[decrypted[i + 1]]
            score += np.log(self.empirical_frequences[first_symbol][second_symbol])

        return score

    def process_decryption(self, encrypted: str, iters: int = 5000, verbose=500) -> str:
        """Process text decryption using MCMC algorithm with random cipher permutations

        Args:
            encrypted (str): encrypted text
            iters (int, optional): _description_. Defaults to 5000.
            verbose (int, optional): _description_. Defaults to 500.

        Returns:
            str: _description_
        """
        assert self.emp_freq_prepared, "Prepare empirical frequences first."
        # Initialize with a random mapping
        shuffled_letters = list(self.alphabet)
        random.shuffle(shuffled_letters)
        current_cifer = dict(zip(shuffled_letters, list(self.alphabet)))
        current_score = self.score_cipher(current_cifer, encrypted)

        best_cifer, best_score = current_cifer.copy(), current_score
        for i in range(0, iters):
            # Create proposal from f by random transposition of 2 letters
            r1, r2 = np.random.choice(list(shuffled_letters), 2, replace=True)
            new_cifer = current_cifer.copy()
            new_cifer[r1] = current_cifer[r2]
            new_cifer[r2] = current_cifer[r1]
            new_score = self.score_cipher(new_cifer, encrypted)

            # Decide to accept new proposal
            if new_score > current_score or random.uniform(0, 1) < np.exp(
                new_score - current_score
            ):
                current_cifer = new_cifer.copy()
                current_score = new_score

            if new_score > best_score:
                best_score = new_score
                best_cifer = new_cifer.copy()

            # Print out progress
            if i % 500 == 0 and verbose:
                assert (
                    type(verbose) is int and verbose > 0
                ), "Select verbose=False or pass positive integer"
                
                if i % verbose == 0:
                    
                    best_attempt_smpl = self.encrypt_or_decrypt(
                        encrypted, 
                        {v: k for k, v in best_cifer.items()}
                    )[:100]
                    
                    print(i, ":\t", best_attempt_smpl)

        # Save best mapping
        cipher_alphabet = "".join([k for k in sorted(best_cifer.keys())])
        plaintext_alphabet = "".join([best_cifer[k] for k in cipher_alphabet])
        mapping = dict(zip(plaintext_alphabet, cipher_alphabet))

        decrypted_text = self.encrypt_or_decrypt(encrypted, mapping)

        return decrypted_text

In [21]:
%%time

enigma = EnigmaForDummies()
enigma.prepare_empirical_freq_normalized("war_and_peace.txt")

plain_text = """
Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?'
So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her.
There was nothing so very remarkable in that; nor did Alice think it so very much out of the way to hear the Rabbit say to itself, 'Oh dear! Oh dear! I shall be late!' (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the Rabbit actually took a watch out of its waistcoat-pocket, and looked at it, and then hurried on, Alice started to her feet, for it flashed across her mind that she had never before see a rabbit with either a waistcoat-pocket, or a watch to take out of it, and burning with curiosity, she ran across the field after it, and fortunately was just in time to see it pop down a large rabbit-hole under the hedge.
In another moment down went Alice after it, never once considering how in the world she was to get out again.
The rabbit-hole went straight on like a tunnel for some way, and then dipped suddenly down, so suddenly that Alice had not a moment to think about stopping herself before she found herself falling down a very deep well.
"""
encrypted = enigma.encrypt_text(plain_text)
print("Encrypted (first 500):\n", encrypted[:500], end="\n\n")

decrypted = enigma.process_decryption(encrypted)
print("\nDecrypted (first 500):\n", decrypted[:500])

Encrypted (first 500):
 dquzrycdvyxrgunnungyloygrlyirwsyluwreyohyvullungyxsymrwyvuvlrwyonylmryxdnjydneyohymdiungynolmungyloyeoyonzryowylcuzryvmrymdeyfrrfreyunloylmryxoojymrwyvuvlrwycdvywrdeungyxblyulymdeynoyfuzlbwrvyowyzonirwvdluonvyunyulydneycmdlyuvylmrybvryohydyxoojylmobgmlydquzryculmoblyfuzlbwrvyowyzonirwvdluonvoyvmrycdvyzonvuerwungyunymrwyocnypuneydvycrqqydvyvmryzobqeyhowylmrymolyedsypderymrwyhrrqyirwsyvqrrfsydneyvlbfueycmrlmrwylmryfqrdvbwryohypdjungydyeduvszmdunycobqeyxrycowlmylmrylwobxqryohygrllungybfydneyfuzjung

0 :	 teijm ptv lmyiooioy zw ymz hmnk zinmf wx vizzioy lk cmn vivzmn wo zcm ltod tof wx cthioy owzcioy zw 
500 :	 udicr hus frnieeien ty nrt prak tiarg yl sittien fk ora sistra ye tor fuev ueg yl oupien eytoien ty 
1000 :	 osige cor pefillilf ta fet menk tined ay rittilf pk hen rirten al the polv old ay homilf lathilf ta 
1500 :	 alice kas beginning to get wery tired of sitting by her sister on the banv and of hawing nothing to 
2000 :	 alice was beginning to get very ti

**As you can see, decoder stabilized after 1500-2000 iterations and provided a good result in the end. Whole process including parsing war_and_peace took only 13.3 seconds to complete on my laptop (hardware dependent though).**  
_NB: convergence speed of MCMC depends on initiation luck. So it may be needed to relaunch the decoder a couple of times to achieve best result_